In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import datetime
import numpy as np
import time
import random

In [2]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import datetime
import numpy as np
import time
import random
# 定义标签集合
tags = {
    "reggae", 
    "rock", "hip-hop", "indie",
    "jazz", "electronic", "british", "punk", "80s", "dance",
    "acoustic", "rnb", "metal", "rap", "classical", "alternative", "blues", "country", "hardcore"
}

album_all = []

for tag in tags:
    albums = []
    distinct = set()
    
    for page in range(1, 30):
        url = f"https://www.last.fm/tag/{tag}/albums?page={page}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch {url}: {response.status_code}")
            continue
        soup = BeautifulSoup(response.content, 'html.parser')
        items = soup.find_all('li', class_='resource-list--release-list-item-wrap')

        for item in items:
            album_info = {}

            # Extract album name
            album_info['name'] = item.find('h3', class_='resource-list--release-list-item-name').a.text

            # Extract album href
            album_info['url'] = "https://www.last.fm" + item.find('h3', class_='resource-list--release-list-item-name').a['href']
            if album_info['url'] in distinct:
                continue
            else: 
                distinct.add(album_info['url'])

            # Extract artist name
            album_info['artist'] = item.find('p', class_='resource-list--release-list-item-artist').span.a.text

            # Extract number of listeners
            listeners = int(item.find('p', class_='resource-list--release-list-item-listeners').text.strip().split(" ")[0].replace(",",""))
            
            if listeners < 1000 : continue
            album_info['listeners'] = listeners

            # Extract release date and number of tracks
            if len(item.find_all('p', class_='resource-list--release-list-item-aux-text'))>1:
                aux_text = item.find_all('p', class_='resource-list--release-list-item-aux-text')[1].text.strip()

                if "·" in aux_text:
                    release_date, tracks = aux_text.split('·')
                    tracks = tracks.split("t")[0]
                
                    album_info['release_date'] = release_date.strip()
                    album_info['tracks'] = tracks.strip()
                else:continue
            else:continue

            album_info['tag'] = tag
            albums.append(album_info)

    # 随机化列表
    random.shuffle(albums)

    # 筛选最多260个专辑
    if len(albums) > 270:
        albums = albums[:270]
    
    album_all.extend(albums)

# 将数据保存为CSV文件
df = pd.DataFrame(album_all)
# df.to_csv("D:\djl\LeetCodes\songs.csv", index=False)
# # album_all

df

,name,url,artist,listeners,release_date,tracks,tag
0,Reach for the Sun,https://www.last.fm/music/The+Dangerous+Summer...,The Dangerous Summer,109963,4 May 2009,2,rock
1,Magic,https://www.last.fm/music/Bruce+Springsteen/Magic,Bruce Springsteen,256754,25 Sep 2007,12,rock
2,On the Way to Wonderland,https://www.last.fm/music/Sunrise+Avenue/On+th...,Sunrise Avenue,254624,1 Jan 2006,15,rock
3,Tal Bachman,https://www.last.fm/music/Tal+Bachman/Tal+Bachman,Tal Bachman,427635,1998,12,rock
4,Evanescence (Deluxe Version),https://www.last.fm/music/Evanescence/Evanesce...,Evanescence,89317,1 Jan 2011,16,rock
...,...,...,...,...,...,...,...
5125,IITE COOL,https://www.last.fm/music/DPR+LIVE/IITE+COOL,DPR LIVE,127406,22 Jul 2021,6,hip-hop
5126,LOST IN PARADISE,https://www.last.fm/music/Ali/LOST+IN+PARADISE,Ali,176712,25 Nov 2020,4,hip-hop
5127,?,https://www.last.fm/music/XXXTENTACION/%3F,XXXTENTACION,1369629,14 Mar 2018,18,hip-hop
5128,IDK & FRIENDS :),https://www.last.fm/music/IDK/IDK+&+FRIENDS+:),IDK,202070,8 Nov 2018,7,hip-hop
